In [1]:
import numpy as np
import sympy as sp

# Calculating the Bhattacharryya bound on compare it with ML decoding

This code is provided as supplementary material of the lecture Channel Coding - Graph-based Codes.

This code illustrates
* Calculating the Bhattacharryya bound on compare it with ML decoding for a specific code

In [2]:
# specify code by its generator matrix
G = np.array([[1, 0, 0, 0, 1, 1, 0],
              [0, 1, 0, 0, 1, 0, 1],
              [0, 0, 1, 0, 0, 1, 1]])

In [3]:
# get parameters k and n of the code
k = G.shape[0]
n = G.shape[1]

# generate table of all codewords
# GF(2) operations are obtained by calculating modulo-2
u = np.array([[(i >> (k - 1 - bit)) & 1 for bit in range(k)] for i in range(2**k)])
code = (u @ G) % 2

In [9]:
# compute weights
w = np.sum(code, 1)#.reshape(-1, 1)
dmin = np.min(w[2:], 0)

# weight enumerator polynomial
A = np.bincount(w, minlength=n+1)[1:]
A = np.append(np.flip(A), 0)

In [5]:
print('Weight enumerator polynomial')
print(sp.Poly(A, sp.symbols("W")).as_expr())

Weight enumerator polynomial
3*W**4 + 4*W**3


In [6]:
# channel
esno_dB = 5 # specify Es/N0 in dB

esno_lin = 10**(esno_dB/10)
# Bhattacharyya parameter
B = np.exp(-esno_lin)

In [7]:
# compute bound on ML performance
P_ML_Bound = np.polyval(A, B)

print(f"According to the Bhattacharyya parameter, the ML error probability is upper bounded by {P_ML_Bound:.4g}\n")

According to the Bhattacharyya parameter, the ML error probability is upper bounded by 0.000313



In [ ]:
# calculate Monte-Carlo-Estimate of ML performance
# transmit N codewords
N = 1000000

# standard deviation of noise distribution
sigma_n = np.sqrt(1 /(2*esno_lin))

errors = 0
for j in range(N):
    # select random codeword
    idx = np.random.randint(0, code.shape[0])
    c = code[idx]
    
    # after channel
    y = (1-2*c) + sigma_n * np.random.randn(c.shape[0])
    
    # calculate correlation with all codewords
    correlation = np.sum((1-2*code)* y, 1)
    
    ML_decision_idx = np.argmax(correlation)
    
    errors = errors + int(ML_decision_idx != idx)

P_ML = errors/N
print(f"After ML decoding, the error rate is approximately {P_ML:4g}\n")

0.3976353643835253
After ML decoding, the error rate is approximately 1.8e-05

